In [1]:
import requests
import json
import pandas as pd

In [41]:
pd.options.mode.chained_assignment = None  # default='warn'
#metadata_staging = pd.read_csv("https://raw.githubusercontent.com/EconomicsObservatory/api/main/metadata-edge.csv")
metadata_staging = pd.read_csv("https://raw.githubusercontent.com/EconomicsObservatory/api/main/metadata-staging.csv")
#metadata_staging = pd.read_csv("/Users/finn/Documents/GitHub/api/metadata-edge.csv")
filtered_df = metadata_staging[metadata_staging.localAPI.str.contains("github")]
github_json_urls = filtered_df["localAPI"].tolist()
country_codes = filtered_df["country"].tolist()

def is_numeric(val):
    try:
        float(val)  # Try to convert the value to float
        return True
    except ValueError:
        return False  # If conversion fails, it is not a numeric value

#check for countries that should be there:
"""UK
USA
Canada
Australia

China
India
Indonesia
Pakistan
Brazil
Nigeria
Bangladesh
Russia
Mexico
Japan
Ethiopia
Philippines
Egypt
Vietnam
DR Congo
Turkey
Iran
Germany
France
Thailand"""


targets = ["gbr", "usa", "can", "aus", "chn", "ind", "idn", "pak", "bra", "nga", "bgd", "rus", "mex", "jpn", "deu", "tha", "eth", "egy", "irn", "tur", "phl", "cod", "vnm", "fra"]
for target in targets:
    # get the len of rows with country code
    country_len = len(metadata_staging[metadata_staging["country"]==target])
    if country_len>0:
        print(f"✅{target}: {country_len} rows")
    else:
        print(f"❌{target}: {country_len} rows")
print("")


# check for duplicates
# find duplicate country, ecoSeries pairs
metadata_staging["country_ecoSeries"] = metadata_staging["country"] + metadata_staging["ecoSeries"]
metadata_staging["duplicate"] = metadata_staging.duplicated(subset="country_ecoSeries", keep=False)
duplicate_df = metadata_staging[metadata_staging["duplicate"]==True]
print(f"Number of duplicate country, ecoSeries pairs: {len(duplicate_df)}")
print(f"Duplicate country, ecoSeries pairs: {duplicate_df['country_ecoSeries'].tolist()}")
print("")

# find missing titles
metadata_staging["country_ecoSeries"] = metadata_staging["country"] + metadata_staging["ecoSeries"]
metadata_staging["missing_title"] = metadata_staging["Title"].isnull()
metadata_staging["missing_subtitle"] = metadata_staging["SubTitle"].isnull()
print(f"Number of missing titles: {metadata_staging['missing_title'].sum()}")
print(f"Number of missing subtitles: {metadata_staging['missing_subtitle'].sum()}")
print("")
print(f"Missing titles: {metadata_staging[metadata_staging['missing_title']==True]['country_ecoSeries'].tolist()}")
print(f"Missing subtitles: {metadata_staging[metadata_staging['missing_subtitle']==True]['country_ecoSeries'].tolist()}")
print("")

# find use_cache == 0
print(f"Number of use_cache == 0: {filtered_df['use_cache'].sum()}")
filtered_df["country_ecoSeries"] = filtered_df["country"] + filtered_df["ecoSeries"]
print(f"use_cache==0: {filtered_df[filtered_df['use_cache']==0]['country_ecoSeries'].tolist()}")
print("")


gh_dfs = []
for country, url in zip(country_codes, github_json_urls):
    print(url, end=" ")
    try:
        r = requests.get(url)
        df = pd.DataFrame(r.json())
        print("✅exists", end=" ")
    except:
        print("❌exists")
        continue
    # check that penultimate portion of url matches country code
    try:
        assert url.split("/")[-2]==country
        print("✅iso3", end=" ")
    except:
        print("❌iso3", end=" ")
    try:
        assert df.columns.to_list()==["date", "value"]
        print("✅columns", end=" ")
    except:
        print("❌columns", end=" ")
    try:
        sample_date = df.date.values[0]
        # assert that date follows yyyy-mm-dd format
        assert len(sample_date)==10
        assert sample_date[4]=="-"
        assert sample_date[7]=="-"
        print("✅date", end=" ")
    except:
        print("❌date", end=" ")
    # check for nan values
    try:
        assert df.isnull().sum().sum()==0
        # check for non-numeric values in value column by looking for cells with non-numbers (except , and a single - or .)
        assert df.value.apply(lambda x: not is_numeric(x)).sum()==0
        print("✅nan", end=" ")
    except:
        print("❌nan", end=" ")
    print("")
    gh_dfs.append(df)


✅gbr: 12 rows
✅usa: 10 rows
✅can: 10 rows
✅aus: 4 rows
✅chn: 7 rows
✅ind: 10 rows
✅idn: 12 rows
✅pak: 10 rows
✅bra: 5 rows
✅nga: 8 rows
✅bgd: 9 rows
✅rus: 7 rows
✅mex: 7 rows
✅jpn: 10 rows
✅deu: 8 rows
✅tha: 10 rows
✅eth: 10 rows
✅egy: 9 rows
✅irn: 6 rows
✅tur: 10 rows
✅phl: 9 rows
✅cod: 10 rows
✅vnm: 9 rows
✅fra: 9 rows

Number of duplicate country, ecoSeries pairs: 0
Duplicate country, ecoSeries pairs: []

Number of missing titles: 0
Number of missing subtitles: 0

Missing titles: []
Missing subtitles: []

Number of use_cache == 0: 156
use_cache==0: []

https://raw.githubusercontent.com/EconomicsObservatory/api/main/cached_data/bgd/infl.json ✅exists ✅iso3 ✅columns ✅date ✅nan 
https://raw.githubusercontent.com/EconomicsObservatory/api/main/cached_data/bgd/grow.json ✅exists ✅iso3 ✅columns ✅date ✅nan 
https://raw.githubusercontent.com/EconomicsObservatory/api/main/cached_data/bgd/gdpa.json ✅exists ✅iso3 ✅columns ✅date ✅nan 
https://raw.githubusercontent.com/EconomicsObservatory/api/main

In [3]:
metadata_staging.query("country == 'tur'")

,country_full,country,ecoSeries,ecoAPI,localAPI,localSeries,localDataset,Title,SubTitle,API_base,...,multiplier,date_keys,value_keys,filter,orient,use_cache,country_ecoSeries,duplicate,missing_title,missing_subtitle
171,Turkiye,tur,empd,https://api.economicsobservatory.com/tur/empd,https://raw.githubusercontent.com/EconomicsObs...,NaN,NaN,Employed,Employed | TURKSTAT through ECO API,https://raw.githubusercontent.com/EconomicsObs...,...,1,NaN,NaN,NaN,NaN,1,turempd,False,False,False
172,Turkiye,tur,empl,https://api.economicsobservatory.com/tur/empl,https://raw.githubusercontent.com/EconomicsObs...,NaN,NaN,Employment Rate,Employment Rate | TURKSTAT through ECO API,https://raw.githubusercontent.com/EconomicsObs...,...,1,NaN,NaN,NaN,NaN,1,turempl,False,False,False
173,Turkiye,tur,expo,https://api.economicsobservatory.com/tur/expo,https://raw.githubusercontent.com/EconomicsObs...,NaN,NaN,Exports,Exports | TURKSTAT through ECO API,https://raw.githubusercontent.com/EconomicsObs...,...,1,NaN,NaN,NaN,NaN,1,turexpo,False,False,False
174,Turkiye,tur,grow,https://api.economicsobservatory.com/tur/grow,https://raw.githubusercontent.com/EconomicsObs...,NaN,NaN,Growth,Growth | TURKSTAT through ECO API,https://raw.githubusercontent.com/EconomicsObs...,...,1,NaN,NaN,NaN,NaN,1,turgrow,False,False,False
175,Turkiye,tur,impo,https://api.economicsobservatory.com/tur/impo,https://raw.githubusercontent.com/EconomicsObs...,NaN,NaN,Imports,Imports | TURKSTAT through ECO API,https://raw.githubusercontent.com/EconomicsObs...,...,1,NaN,NaN,NaN,NaN,1,turimpo,False,False,False
176,Turkiye,tur,inac,https://api.economicsobservatory.com/tur/inac,https://raw.githubusercontent.com/EconomicsObs...,NaN,NaN,Inactivity,Inactivity | TURKSTAT through ECO API,https://raw.githubusercontent.com/EconomicsObs...,...,1,NaN,NaN,NaN,NaN,1,turinac,False,False,False
177,Turkiye,tur,ineq,https://api.economicsobservatory.com/tur/ineq,https://raw.githubusercontent.com/EconomicsObs...,NaN,NaN,Inequality,Inequality | TURKSTAT through ECO API,https://raw.githubusercontent.com/EconomicsObs...,...,1,NaN,NaN,NaN,NaN,1,turineq,False,False,False
178,Turkiye,tur,infl,https://api.economicsobservatory.com/tur/infl,https://raw.githubusercontent.com/EconomicsObs...,NaN,NaN,Inflation,Inflation | TURKSTAT (WB) through ECO API,https://raw.githubusercontent.com/EconomicsObs...,...,1,NaN,NaN,NaN,NaN,1,turinfl,False,False,False
179,Turkiye,tur,popu,https://api.economicsobservatory.com/tur/popu,https://raw.githubusercontent.com/EconomicsObs...,NaN,NaN,Population,Population | TURKSTAT through ECO API,https://raw.githubusercontent.com/EconomicsObs...,...,1,NaN,NaN,NaN,NaN,1,turpopu,False,False,False
180,Turkiye,tur,unem,https://api.economicsobservatory.com/tur/unem,https://raw.githubusercontent.com/EconomicsObs...,NaN,NaN,Unemployment Rate,Unemployment Rate | TURKSTAT through ECO API,https://raw.githubusercontent.com/EconomicsObs...,...,1,NaN,NaN,NaN,NaN,1,turunem,False,False,False


In [4]:
cached = filtered_df.country_full.unique()
wb = ['Ethiopia', 'Egypt', 'India', 'Japan', 'Thailand', 'Pakistan', 'Nigeria', 'Bangladesh', 'Vietnam', 'DR Congo']
not_wb = [c for c in cached if c not in wb]
print(not_wb)
len(not_wb)

metadata_staging = pd.read_csv("https://raw.githubusercontent.com/EconomicsObservatory/api/main/metadata-staging.csv")
not_cached = [c for c in metadata_staging.country_full.unique() if c not in cached]
print(not_cached)
print(len(not_cached))

['Brazil', 'China', 'France', 'Germany', 'Iran', 'Philippines', 'Turkiye', 'Russia']
['Australia', 'Canada', 'Great Britain', 'Indonesia', 'Mexico', 'USA']
6


In [21]:
filtered_df

,country_full,country,ecoSeries,ecoAPI,localAPI,localSeries,localDataset,Title,SubTitle,API_base,...,units,date,field,multiplier,date_keys,value_keys,filter,orient,use_cache,country_ecoSeries
4,Bangladesh,bgd,debt,https://api.economicsobservatory.com/bgd/debt,https://raw.githubusercontent.com/EconomicsObs...,NaN,NaN,Debt,"Central government debt, total (% of GDP) | BB...",https://raw.githubusercontent.com/EconomicsObs...,...,% of GDP,NaN,NaN,1,NaN,NaN,NaN,NaN,1,bgddebt
5,Bangladesh,bgd,empl,https://api.economicsobservatory.com/bgd/empl,https://raw.githubusercontent.com/EconomicsObs...,NaN,NaN,Employment to population ratio,"Employment to population ratio, 15+, total (%)...",https://raw.githubusercontent.com/EconomicsObs...,...,%,NaN,NaN,1,NaN,NaN,NaN,NaN,1,bgdempl
6,Bangladesh,bgd,expo,https://api.economicsobservatory.com/bgd/expo,https://raw.githubusercontent.com/EconomicsObs...,NaN,NaN,Exports,Exports of goods and services (current US$) | ...,https://raw.githubusercontent.com/EconomicsObs...,...,current US$,NaN,NaN,1,NaN,NaN,NaN,NaN,1,bgdexpo
7,Bangladesh,bgd,gdpa,https://api.economicsobservatory.com/bgd/gdpa,https://raw.githubusercontent.com/EconomicsObs...,NaN,NaN,GDP,GDP (constant 2015 US$) | BBS (WB) through ECO...,https://raw.githubusercontent.com/EconomicsObs...,...,constant 2015 US$,NaN,NaN,1,NaN,NaN,NaN,NaN,1,bgdgdpa
8,Bangladesh,bgd,gdpp,https://api.economicsobservatory.com/bgd/gdpp,https://raw.githubusercontent.com/EconomicsObs...,NaN,NaN,GDP per capita,GDP per capita (constant 2015 US$) | BBS (WB) ...,https://raw.githubusercontent.com/EconomicsObs...,...,constant 2015 US$,NaN,NaN,1,NaN,NaN,NaN,NaN,1,bgdgdpp
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
212,Russia,rus,unem,https://api.economicsobservatory.com/rus/unem,https://raw.githubusercontent.com/EconomicsObs...,NaN,NaN,Unemployment Rate,Unemployment Rate | Rosstat through ECO API,https://raw.githubusercontent.com/EconomicsObs...,...,NaN,date,value,1,NaN,NaN,NaN,NaN,1,rusunem
213,Russia,rus,empd,https://api.economicsobservatory.com/rus/empd,https://raw.githubusercontent.com/EconomicsObs...,NaN,NaN,Employed,Employed | Rosstat through ECO API,https://raw.githubusercontent.com/EconomicsObs...,...,NaN,date,value,1,NaN,NaN,NaN,NaN,1,rusempd
214,Russia,rus,umpd,https://api.economicsobservatory.com/rus/umpd,https://raw.githubusercontent.com/EconomicsObs...,NaN,NaN,Unemployed,Unemployed | Rosstat through ECO API,https://raw.githubusercontent.com/EconomicsObs...,...,NaN,date,value,1,NaN,NaN,NaN,NaN,1,rusumpd
215,Russia,rus,expo,https://api.economicsobservatory.com/rus/expo,https://raw.githubusercontent.com/EconomicsObs...,NaN,NaN,Exports,Exports | Rosstat through ECO API,https://raw.githubusercontent.com/EconomicsObs...,...,NaN,date,value,1,NaN,NaN,NaN,NaN,1,rusexpo


In [42]:
filtered_df["series_from_localAPI"] = filtered_df["localAPI"].apply(lambda x: x.split("/")[-1].split(".json")[0])
filtered_df[filtered_df["series_from_localAPI"]!=filtered_df["ecoSeries"]]

,country_full,country,ecoSeries,ecoAPI,localAPI,localSeries,localDataset,Title,SubTitle,API_base,...,date,field,multiplier,date_keys,value_keys,filter,orient,use_cache,country_ecoSeries,series_from_localAPI


In [20]:
problems

[True,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

# Visual Validation

In [26]:
filtered_df["country_full"].tolist()

['Bangladesh',
 'Bangladesh',
 'Bangladesh',
 'Bangladesh',
 'Bangladesh',
 'Bangladesh',
 'Bangladesh',
 'Bangladesh',
 'Bangladesh',
 'Bangladesh',
 'Brazil',
 'Brazil',
 'Brazil',
 'Brazil',
 'Brazil',
 'Brazil',
 'China',
 'China',
 'China',
 'China',
 'China',
 'China',
 'China',
 'Egypt',
 'Egypt',
 'Egypt',
 'Egypt',
 'Egypt',
 'Egypt',
 'Egypt',
 'Egypt',
 'Egypt',
 'Egypt',
 'Ethiopia',
 'Ethiopia',
 'Ethiopia',
 'Ethiopia',
 'Ethiopia',
 'Ethiopia',
 'Ethiopia',
 'Ethiopia',
 'Ethiopia',
 'Ethiopia',
 'France',
 'France',
 'France',
 'France',
 'France',
 'France',
 'France',
 'France',
 'France',
 'Germany',
 'Germany',
 'Germany',
 'Germany',
 'Germany',
 'Germany',
 'Germany',
 'Germany',
 'India',
 'India',
 'India',
 'India',
 'India',
 'India',
 'India',
 'India',
 'India',
 'India',
 'Iran',
 'Iran',
 'Iran',
 'Iran',
 'Iran',
 'Iran',
 'Iran',
 'Japan',
 'Japan',
 'Japan',
 'Japan',
 'Japan',
 'Japan',
 'Japan',
 'Japan',
 'Japan',
 'Japan',
 'Nigeria',
 'Nigeria',
 '

In [28]:
if input("redraw? y/n")!="y":
    raise Exception("Aborted")

import altair as alt
from IPython.core.display import display, HTML
import ipywidgets as widgets

full_country_names = filtered_df["country_full"].tolist()
titles = filtered_df["Title"].tolist()
subtitles = filtered_df["SubTitle"].tolist()

ecoSeries = filtered_df["ecoSeries"].tolist()
iso3 = filtered_df["country"].tolist()

problems = [0]*len(full_country_names)
deletes = [False]*len(full_country_names)
checkboxes = []
problem_inputs = []

# This function will be called when a checkbox state is changed
def on_checkbox_change(change, index):
    change = int(change['new'])
    problems[index] = change
    # append new line to problem_log.csv with country, ecoSeries, problem_status
    with open("problem_log.csv", "a") as f:
        f.write(f"{iso3[index]},{ecoSeries[index]},{change}\n")

def on_delete_checkbox_change(change, index):
    deletes[index] = change['new']
    # append new line to delete_log.csv with country, ecoSeries, delete_status
    with open("delete_log.csv", "a") as f:
        f.write(f"{iso3[index]},{ecoSeries[index]},{change['new']}\n")

def on_problem_input_change(change, index):
    problem = change['new']
    # write problem description to problem_log.csv
    with open("problem_log.csv", "a") as f:
        f.write(f"{iso3[index]},{ecoSeries[index]},\"{problem}\"\n")

start_index = 0
for i, df in enumerate(gh_dfs[start_index:]):
    i = i + start_index
    print(full_country_names[i])
    checkbox = widgets.Checkbox(value=False, description='Problem', disabled=False)
    checkbox.observe(lambda change, index=i: on_checkbox_change(change, index), names='value')
    checkboxes.append(checkbox)
    
    problem_input = widgets.Text(value='', placeholder='Enter problem description', description='Problem:', disabled=False)
    problem_input.observe(lambda change, index=i: on_problem_input_change(change, index), names='value')
    problem_inputs.append(problem_input)
    
    delete_checkbox = widgets.Checkbox(value=False, description='Delete', disabled=False)
    delete_checkbox.observe(lambda change, index=i: on_delete_checkbox_change(change, index), names='value')
    
    chart = alt.Chart(df).mark_line().encode(
        x=alt.X('date:T', axis=alt.Axis(title=None, format='%b %Y')),
        y='value:Q'
    ).properties(width=320, height=250,
        title=alt.TitleParams(
            titles[i],
            subtitle=subtitles[i],
            anchor='middle',
            fontSize=16,
            fontWeight='normal'
        )
    )
    # Display the chart, problem input, and delete checkbox without using a VBox
    display(chart)
    display(problem_input)
    display(delete_checkbox)
    display(checkbox)

Bangladesh


/var/folders/07/g_yq2xsn0mvb4xnyvr18jjmw0000gn/T/ipykernel_71612/3206612407.py:5: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


alt.Chart(...)

Text(value='', description='Problem:', placeholder='Enter problem description')

Checkbox(value=False, description='Delete')

...

Russia


alt.Chart(...)

Text(value='', description='Problem:', placeholder='Enter problem description')

Checkbox(value=False, description='Delete')

Checkbox(value=False, description='Problem')

In [27]:
if input("redraw? y/n")!="y":
    raise Exception("Aborted")

import altair as alt
from IPython.core.display import display, HTML
import ipywidgets as widgets

full_country_names = filtered_df["country_full"].tolist()
titles = filtered_df["Title"].tolist()
subtitles = filtered_df["SubTitle"].tolist()

ecoSeries = filtered_df["ecoSeries"].tolist()
iso3 = filtered_df["country"].tolist()

problems = [0]*len(full_country_names)
deletes = [False]*len(full_country_names)
checkboxes = []
problem_inputs = []

# This function will be called when a checkbox state is changed
def on_checkbox_change(change, index):
    change = int(change['new'])
    problems[index] = change
    # append new line to problem_log.csv with country, ecoSeries, problem_status
    with open("problem_log.csv", "a") as f:
        f.write(f"{iso3[index]},{ecoSeries[index]},{change}\n")

def on_delete_checkbox_change(change, index):
    deletes[index] = change['new']
    # append new line to delete_log.csv with country, ecoSeries, delete_status
    with open("delete_log.csv", "a") as f:
        f.write(f"{iso3[index]},{ecoSeries[index]},{change['new']}\n")

def on_problem_input_change(change, index):
    problem = change['new']
    # write problem description to problem_log.csv
    with open("problem_log.csv", "a") as f:
        f.write(f"{iso3[index]},{ecoSeries[index]},\"{problem}\"\n")

start_index = 96
for i, df in enumerate(gh_dfs[96:]):
    i = i + start_index
    print(full_country_names[i])
    checkbox = widgets.Checkbox(value=False, description='Problem', disabled=False)
    checkbox.observe(lambda change, index=i: on_checkbox_change(change, index), names='value')
    checkboxes.append(checkbox)
    
    problem_input = widgets.Text(value='', placeholder='Enter problem description', description='Problem:', disabled=False)
    problem_input.observe(lambda change, index=i: on_problem_input_change(change, index), names='value')
    problem_inputs.append(problem_input)
    
    delete_checkbox = widgets.Checkbox(value=False, description='Delete', disabled=False)
    delete_checkbox.observe(lambda change, index=i: on_delete_checkbox_change(change, index), names='value')
    
    chart = alt.Chart(df).mark_line().encode(
        x=alt.X('date:T', axis=alt.Axis(title=None, format='%b %Y')),
        y='value:Q'
    ).properties(width=320, height=250,
        title=alt.TitleParams(
            titles[i],
            subtitle=subtitles[i],
            anchor='middle',
            fontSize=16,
            fontWeight='normal'
        )
    )
    # Display the chart, problem input, and delete checkbox without using a VBox
    display(chart)
    display(problem_input)
    display(delete_checkbox)
    display(checkbox)

Pakistan


/var/folders/07/g_yq2xsn0mvb4xnyvr18jjmw0000gn/T/ipykernel_71612/1673737810.py:5: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


alt.Chart(...)

Text(value='', description='Problem:', placeholder='Enter problem description')

Checkbox(value=False, description='Delete')

...

Russia


alt.Chart(...)

Text(value='', description='Problem:', placeholder='Enter problem description')

Checkbox(value=False, description='Delete')

Checkbox(value=False, description='Problem')

In [14]:
import altair_viewer
altair_viewer.ChartViewer(gh_dfs[0].to_dict())

In [24]:
df = gh_dfs[0]
df.value.astype(str).str.strip().contains("[^0-9\.,-]").sum()

AttributeError: 'Series' object has no attribute 'contains'

In [28]:
df.value.astype(str).str.contains("[^0-9\.,-]").sum()

0

In [11]:
metadata_staging

,country_full,country,ecoSeries,ecoAPI,localAPI,localSeries,localDataset,Title,SubTitle,API_base,...,description,units,date,field,multiplier,date_keys,value_keys,filter,orient,use_cache
0,Great Britain,gbr,debt,https://api.economicsobservatory.com/gbr/debt,https://api.ons.gov.uk/timeseries/HF6X/dataset...,HF6X,PUSF,Debt,"Public sector net debt, % GDP | Source: ONS th...",https://api.ons.gov.uk/timeseries/{SERIES}/dat...,...,"PS, Net Debt as a % of GDP, NSA",% of GDP,months.date,months.value,1,NaN,NaN,NaN,NaN,0
1,Great Britain,gbr,infl,https://api.economicsobservatory.com/gbr/infl,https://api.ons.gov.uk/timeseries/L55O/dataset...,L55O,MM23,Inflation,"All items, % | Source: ONS through ECO API",https://api.ons.gov.uk/timeseries/{SERIES}/dat...,...,"CPIH annual rate, All items, 2015=100",%,months.date,months.value,1,NaN,NaN,NaN,NaN,0
2,Great Britain,gbr,grow,https://api.economicsobservatory.com/gbr/grow,https://api.ons.gov.uk/timeseries/IHYR/dataset...,IHYR,QNA,Growth,% | Source: ONS through ECO API,https://api.ons.gov.uk/timeseries/{SERIES}/dat...,...,"Gross Domestic Product, QoQ4 growth rate, CVM SA",%,quarters.date,quarters.value,1,NaN,NaN,NaN,NaN,0
3,Great Britain,gbr,unem,https://api.economicsobservatory.com/gbr/unem,https://api.ons.gov.uk/timeseries/MGSX/dataset...,MGSX,LMS,Unemployment rate,"Ages 16 and over, % | Source: ONS through ECO API",https://api.ons.gov.uk/timeseries/{SERIES}/dat...,...,"Unemployment rate, ages 16 and over, seasonall...",%,months.date,months.value,1,NaN,NaN,NaN,NaN,0
4,Great Britain,gbr,empl,https://api.economicsobservatory.com/gbr/empl,https://api.ons.gov.uk/timeseries/MGSR/dataset...,MGSR,LMS,Employment rate,"Ages 16 and over, % | Source: ONS through ECO API",https://api.ons.gov.uk/timeseries/{SERIES}/dat...,...,NaN,%,months.date,months.value,1,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
156,India,ind,unem,https://api.economicsobservatory.com/ind/unem,https://raw.githubusercontent.com/EconomicsObs...,NaN,NaN,Unemployment,"Unemployment, total (% of total labor force) (...",https://raw.githubusercontent.com/EconomicsObs...,...,"Unemployment, total (% of total labor force) (...",% of total labor force,NaN,NaN,1,NaN,NaN,NaN,NaN,1
157,India,ind,empl,https://api.economicsobservatory.com/ind/empl,https://raw.githubusercontent.com/EconomicsObs...,NaN,NaN,Employment to population ratio,"Employment to population ratio, 15+, total (%)...",https://raw.githubusercontent.com/EconomicsObs...,...,"Employment to population ratio, 15+, total (%)...",%,NaN,NaN,1,NaN,NaN,NaN,NaN,1
158,India,ind,expo,https://api.economicsobservatory.com/ind/expo,https://raw.githubusercontent.com/EconomicsObs...,NaN,NaN,Exports,Exports of goods and services (current US$) | ...,https://raw.githubusercontent.com/EconomicsObs...,...,Exports of goods and services (current US$),current US$,NaN,NaN,1,NaN,NaN,NaN,NaN,1
159,India,ind,impo,https://api.economicsobservatory.com/ind/impo,https://raw.githubusercontent.com/EconomicsObs...,NaN,NaN,Imports,Imports of goods and services (current US$) | ...,https://raw.githubusercontent.com/EconomicsObs...,...,Imports of goods and services (current US$),current US$,NaN,NaN,1,NaN,NaN,NaN,NaN,1


,country_full,country,ecoSeries,ecoAPI,localAPI,localSeries,localDataset,Title,SubTitle,API_base,...,date,field,multiplier,date_keys,value_keys,filter,orient,use_cache,country_ecoSeries,duplicate


In [31]:
sample_date = gh_dfs[0].date.values[0]
# assert that date follows yyyy-mm-dd format
assert len(sample_date)==10
assert sample_date[4]=="-"
assert sample_date[7]=="-"

In [ ]:
metadata_staging.to_csv("temp.csv", index=False)

gh_dfs